In [1]:
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from ACERNetwork import * 
from Worker import * 
import gym as gym

network_t = "cnn"
GAME_NAME="Breakout-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/breakout/test/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 8e-5
print(env.action_space)

/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Discrete(4)


/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [2]:
from tensorflow import Session, ConfigProto
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
config = ConfigProto()
config.gpu_options.allow_growth=True
Session(config=config).close()

In [3]:
session = tf.Session()
tf.reset_default_graph()

with tf.Session() as sess:
    with tf.device('/cpu:0'):
        agent = ACERNetwork(sess,env,str(0), network_type=network_t)
        sess.run(tf.global_variables_initializer())
        print(agent.get_pi(state))

Tensor("net_0/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_0/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_0/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)


/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[0.24725494 0.24149354 0.26520258 0.2460489 ]]


In [4]:
def start_method(game_name=GAME_NAME, num_agents = 16, restore = False, save_path=TB_DIR, replay_ratio=4,network_type="cnn"):
        
    envs=[]
    
    for _ in range(num_agents):
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
            agent=ACERNetwork(sess,envs[0],"agent",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            saver = tf.train.Saver(max_to_keep=5)
            
            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())

                
            worker = Worker(agent,envs,sess,"eval", 20, 0.99, 250000, replay_ratio)
            worker.work_acer()


In [5]:
tf.reset_default_graph()
start_method(replay_ratio=0)

Tensor("net_agent/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32)
Tensor("net_agent/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32)
Tensor("net_agent/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32)


/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


worker_eval  using  0  per online step


/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


KeyboardInterrupt: 

In [ ]:
TB_DIR = "saves/spaceinv/final_R1/"
tf.reset_default_graph()
start_method(replay_ratio=1)

In [ ]:
def test_method(game_name=GAME_NAME, num_agents = 1, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=False)
        envs.append(a_env)
       


    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=2)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
                
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=test, args=(agents[thread_id],
                envs[thread_id], 10, True)))

            for p in processes:
                p.daemon = True
                p.start()

                
            for p in processes:
                p.join()


In [ ]:
tf.reset_default_graph()
test_method()